In [1]:
import numpy as np
import pandas as pd
import torch
from collections import defaultdict
import esm

In [2]:
reference_id = "9606_0:004caa"
k = 14

In [3]:
seq_dict = {
    "9606_0:004caa":"FAEAWKAQFPDSEPPRMELRSVGDIEQELERCKASIRRLEQEVNQERFRMIYLQTLLAKEKKSYDRQRWGFRRAAQAPDGASEPRASASRPQPAPADGADPPPAEEPEARPDGEGSPGKARPGTARRPGAAASGERDDRGPPASVAALRSNFERIRKGHGQPGADAEKPFYVNVEFHHERGLVKVNDKEVSDRISSLGSQAMQMERKKSQHGAGSSVGDASRPPYRGRSSESSCGVDGDYEDAELNPRFLKDNLIDANGGSRPPWPPLEYQPYQSIYVGGMMEGEGKGPLLRSQSTSEQEKRLTWPRRSYSPRSFEDCGGGYTPDCSSNENLTSSEEDFSSGQSSRVSPSPTTYRMFRDKSRSPSQNSQQSFDSSSPPTPQCHKRHRHCPVVVSEATIVGVRKTGQIWPNDGEGAFHGDADGSFGTPPGYGCAADRAEEQRRHQDGLPYIDDSPSSSPHLSSKGRGSRDALVSGALESTKASE",
    "7757_0:00087f":"FAEAWRAQFPDSEPPKMELRSLRDIEGELCSCKNSIRRLEVEVNKERFRMIYLQTLLAKEKKGYDRQRWGFRKSAQGGGGVAVPGPAGDEAGPDDDGDDPVGEVAAAAAAAAAALGGEMPGGGGGADGERPFPESPSHLRRHAADPDEGFEDTIASSRHRRAKAPDKSPLPPGLVIAPGKGAKGGEPGSPAAAGSPCKSPRVPGPPEPRARRHSSQTSEEAALCGGGGGGSRGSPGADKSSSPLQQQQQLLPPSYANVEFHAERGLVRLNEREVSDHISSLGSEAMRMERRRGGCGTASREPCVSPGRDPPQLRGPSGPGVSASSSSSSGRHHHHHHHQAASHVGYEADYEDAEPPGRFFAASPKQHPKRERPPWHVAEEATGLPESTALTDGDSTYGTGSREQLYPSSGRRQPGTGVPGLGGPPGGGGGGAGFVACKEERLSWPKQQRAHSPSSLEDAGGGYTPDCSSNDNLTSSEDELSSPGGRSGRGGGGGVGGGGAGGGVGMGLAGTGASASPTAGSSSYRLFRERENRSPSQNSQHSLDSSSPPTPHGHGTPAGLGGSGGGGGGGGRRAVRGKPSGGPGAKAVTPPGGGPDSPQRSPAGLGPKRGPPTRSRYGAPDDVFDSYPGESPFGVPSPGYVYDAERGDDGRPPPEGMPYIDDSPSSSPHAAARIRGARDAAAAAAATSAAALDAGKMGE",
    "42100_0:0032b1":"FAEAWKAQFPDSEPPRMELRSVGDIEQELDRCKASIRRLEQEVNQERFRMIYLQTLLAKEKKSYDRQRWGFRRAAQPPDGAAEPRAPRPQPAPADCSDPPPADEPEARLDGEGSPGKARPAATRRPGAAAPADHDDRGPPTSVAALRSNFERIRKGPVQPAATDAEKPFYVNVEFHHERGLVKVNDKDVSDRISSLGSQAMQMERKKSQQGAGPGPGDPPRPSYRGRLSESSCGLDGDFEDAELNPRFLKDNLINANGGSRPPWPPLEYQPYQSIYVGGMMVEGEGKGSLQRSQSTSEQEKRLTWPRRSYSPRSFEDSGGGYTPDCSSNENLTSSEEDFSSGQSSRVSPSPTTYRTFRDKSRSPSQNSQQSFDSSSPPTPQCQKRHRQGPVVVSEATIVGVRKTGQIWPSDGDSLSSRLLQDGAFHGDADAPFGTPPGYGCATDRAEEQRRHQDGLPYMDDSPSSSPHLGSKGRGSRDAPESTKVSE",
    "8255_0:002868":"FVEAWRAQFPESDPPCMELNSLGDIEQELDRCKTSIRQLEKEVNKERFRMIYLQTLLAKERKSYDGQRWGFRRVPQTRDGDQTSEPDSQRSYAGDAPVEEQQPQQQQQQRDYVKPARCKQHPEEEMDGASPAKQKGGVSLVGQDSENPEFPVGTGSVAALRSNFERIRRANSHTAGDGRGLSTMGGQEKPFYVNMEYHHERGLVKVNDRDVSDKIGTLGCQAMQIERKRSLHSLPGNLSAATGDISKAVQKSTEENCSYNGSCDDPEINPHFLKDSVIQPTGGKSERHPAECQPYTSVYVGGMMADGDTRVIHIRDHSIEEEAHLTWPRHSYSPGSFDDVGGGYTPDCSSNENLTSSEEDFSSGQSSHVSPSPTTYQMYREKSRSPSQHSQQSFESSSPPTPLSQKRLKQQVMVSEASIIGVRKTGLPWRSDGDSTTSSRTSHDNSVQGDLEAPYNETPLTYGYELEHSEEHQPHHDPLSYTGSPSSSPRLRSKSRSSRDTQSSGSLESTLSVE",
    "215358_0:000d84":"FVEAWRAQFPESDPPCMDLNSMGDIEQELDKCKTSIRHLEKEVNKERFRMIYLQTLLAKERKSYDGQRWGFKRTPQMNDGDLPTIPDNQRSHTDETAVEEHPLRGPGKTARCKPHPEGEVDGASPAKQKGGVSPVRQDSEAPDFPLGTGSVAALRSNFERIRRDSSHAAGDGRGSSEKPFYVNVEYHHERGLVKVNDREVSDKISTLGCQAMQMERKRSLHSLPGNLSAAMGDINKAVRRGRSTEGNCSYNGSYDDAEANPHFLKDNVIRSTGGKSERQPAECQPYTSVYVGGMMADGDTRIIHIRDHSIEEDAHLTWPRRSYSPGSFDDVGGGYTPDCSSNENLTSSEEDFSSGQSSHVSPSPTTYQMYREKSRSPSQHSFDSSSPPTPLSQKCLKQRAMVSEASIIRGQIWPSDGDSTTSSRTSHDNSLQGDLEAPYNETPPSYGYDLEHSEEHQPRHDPLSFTGSLESTLSVE",
    "13616_0:00197d":"FERHWRAEFPGEPVPAMQLDSVPAMQEELERCKLNLKRLQQVLAEEKFKVIYLQTALAREKRGYDCGRWEGGGGEGDSCSSSGPTKEEEEGVEAEEVRAPTELEHPQPRPQPQPQPRPRSRPQPRPRTRPEPHKEPSSSDSGPGGLASVPRTKSPHYVNLEFPAWRGTPGEGGAVLSLTAAIHQQLQQPQLLLRPREEQEPELELETERALGNGRRPEGSAQSPSALVSESPGASPEEGEELSPAAPEDTEPEALVAAEEGGDSDHDFEVVDLNEKFVLGHLLLAPCRFLPREDTPEKPLRAGSPHRWLHLIGGKRHRRSRDLDQLEAEGKDGRCSPLPPEEAHFRGAREHLPPWRRKKLLGVPGRDSPGHSSPERGSDCSYNSSDHEDVSSADFSYGADDYDGEGNEEQKVPPEGSETMPYIDESPTMSPQLSARSQGSGDGISPTPPEGLGPVVEAG",
    "336983_0:0004f6":"MEPLSHRGLPRLSWIDTLYSNFSYGTDEYDGEGNEEQKGPPEGSETMPYIDESPTMSPQLSARSQGGGDGVSPTPPEGLAPGVEAG",
}

In [4]:
class ESM_Model:
    '''
    This is from the Kibby package and slightly modified
    '''
    # esm1b_t33_650M_UR50S
    # esm2_t6_8M_UR50D
    # esm2_t12_35M_UR50D
    # esm2_t30_150M_UR50D
    # esm2_t33_650M_UR50D
    # esm2_t36_3B_UR50D

    def __init__(self, model_name: str = "esm2_t33_650M_UR50D"):
        self.load(model_name)

    def load(self, model_name):
        import esm

        self.model_name = model_name
        self.model, alphabet = eval(f"esm.pretrained.{self.model_name}()")
        self.batch_converter = alphabet.get_batch_converter()
        self.model.eval()
        self.embed_dim = self.model._modules["layers"][0].embed_dim
        self.layers = sum(1 for i in self.model._modules["layers"])

    def encode(self, sequence, device="cuda", threads=1):
        try:
            torch.cuda.empty_cache()
            torch.set_num_threads(threads)

            batch_labels, batch_strs, batch_tokens = self.batch_converter(
                [["", sequence]]
            )
            batch_tokens = batch_tokens.to(device)
            self.model = self.model.to(device)
            with torch.no_grad():
                results = self.model(
                    batch_tokens, repr_layers=[self.layers], return_contacts=False
                )
                results = results["representations"][self.layers].to("cpu")[0]
            return results
        except:
            if device != "cpu":
                return self.encode(sequence, device="cpu", threads=threads)
            else:
                return

In [5]:
def euclidean_distance(tensor1, tensor2):
    squared_diff = np.square(tensor1 - tensor2)
    sum_squared_diff = np.sum(squared_diff)
    distance = np.sqrt(sum_squared_diff)
    return distance


def make_empty_kmer_ortho_df(positions, ortholog_ids: list[str]):
    cols = ["reference_kmer"] + ortholog_ids
    df = pd.DataFrame(
        index=positions,
        columns=cols,
    )
    return df


def generate_kmer_embeddings(seq: str, tensor, k: int):
    """Generates kmers and their corresponding embeddings from a sequence and
    its tensor
    Assumes that the sequence and tensor are of the same length (i.e. no start
    and end tokens are included in the tensor)"""
    k2 = k - 1
    kmers = []
    kmer_tensors = []
    for i in range(len(seq) - k2):
        kmers.append(seq[i : i + k])
        kmer_tensors.append(tensor[i : i + k, :])
    return kmers, kmer_tensors


def get_embedding_dict(
    sequence_dict: dict[str, str], mod: ESM_Model
):
    '''Returns a dictionary of sequences and their embeddings for each sequence in the input dictionary
    
    input dict: {seq_id: seq_str}
    returned dict: {seq_id: [seq_str, seq_tensor]}
    '''
    embedding_dict = defaultdict(list)
    for seq_id, seq_str in sequence_dict.items():
        seq_tensor = mod.encode(seq_str, device="cuda")[1:-1, :] # remove start and end tokens
        embedding_dict[seq_id].append(seq_str)
        embedding_dict[seq_id].append(seq_tensor)
    return embedding_dict


def find_best_ortho_match_embeddings(kmer_tensor, ortho_str, ortho_tensor):
    """find best match between kmer tensor and an ortholog tensor
    Uses a sliding window approach and calculates the euclidean distance between
    the `kmer_tensor` and each length k slice of the `ortho_tensor`. Returns the
    distance, sequence, and position of the best matching ortholog slice of length k
    
    Notes: don't need to use the sequence, could just use the returned best position to
    slice the sequence later

    Parameters
    ----------
    kmer_tensor : torch.Tensor
        kmer tensor slice
    ortho_str : str
        sequence of the ortholog
    ortho_tensor : torch.Tensor
        full tensor of the ortholog sequence. Will be sliced to the same length
        as the kmer tensor for each position (sliding window approach)

    Returns
    -------
    tuple
        returns a tuple of the (euclidean distance, sequence, position) of 
        the best matching ortholog slice of length k
    """    
    pos, dist, seq = [], [], []
    for i in range(len(ortho_str) - (len(kmer_tensor) - 1)):
        # slice the ortholog tensor to the same length as the kmer tensor
        target_tensor = ortho_tensor[i : i + len(kmer_tensor), :]
        # calculate the euclidean distance between the kmer tensor and the ortholog slice tensor
        dist_i = euclidean_distance(np.array(kmer_tensor), np.array(target_tensor))
        pos.append(i)
        dist.append(dist_i)
        seq.append(ortho_str[i : i + len(kmer_tensor)])
    best_match_ind = np.argmin(dist)
    return dist[best_match_ind], seq[best_match_ind], pos[best_match_ind]

# def find_best_ortho_match_embeddings_v2(kmer_tensor, ortho_str, ortho_tensor):
#     best_dist = np.inf
#     for i in range(len(ortho_str) - (len(kmer_tensor) - 1)):
#         # slice the ortholog tensor to the same length as the kmer tensor
#         target_tensor = ortho_tensor[i : i + len(kmer_tensor), :]
#         # calculate the euclidean distance between the kmer tensor and the ortholog slice tensor
#         dist_i = euclidean_distance(np.array(kmer_tensor), np.array(target_tensor))
#         if dist_i < best_dist:
#             best_dist = dist_i
#             best_pos = i
#     return best_dist, ortho_str[best_pos : best_pos + len(kmer_tensor)], best_pos

In [6]:
def run_pairwise_kmer_emb_aln(
    reference_id: str,
    embedding_dict: dict,
    k: int,
):
    # get the reference sequence and remove it from the embedding_dict
    ref_seq_str, ref_seq_embedding = embedding_dict.pop(reference_id)
    kmers, kmer_tensors = generate_kmer_embeddings(
        seq=ref_seq_str,
        tensor=ref_seq_embedding,
        k=k,
    )
    positions = list(range(len(kmers)))
    score_df = make_empty_kmer_ortho_df(positions, list(embedding_dict.keys()))
    subseq_df = make_empty_kmer_ortho_df(positions, list(embedding_dict.keys()))
    pos_df = make_empty_kmer_ortho_df(positions, list(embedding_dict.keys()))
    
    # for each kmer in the reference sequence
    for position, kmer, kmer_tensor in zip(positions, kmers, kmer_tensors):
        score_df.loc[position, "reference_kmer"] = kmer
        subseq_df.loc[position, "reference_kmer"] = kmer
        pos_df.loc[position, "reference_kmer"] = kmer

        # for each ortholog sequence
        for ortholog_id, v in embedding_dict.items():
            ortholog_seq = v[0]
            ortholog_embedding = v[1]
            if ortholog_seq is None or ortholog_embedding is None:
                continue
            if len(ortholog_seq) < k:
                continue
            try:
                # find the best match between the kmer tensor and the ortholog tensor
                best_score, best_subseq, best_pos = find_best_ortho_match_embeddings(
                    kmer_tensor, ortholog_seq, ortholog_embedding
                )
            except ValueError as e:
                print(e)
                continue
            score_df.loc[position, ortholog_id] = best_score
            subseq_df.loc[position, ortholog_id] = best_subseq
            pos_df.loc[position, ortholog_id] = best_pos
    return score_df, subseq_df, pos_df


def main(
    sequence_dict: dict[str, str],
    reference_id: str,
    k: int,
):
    mod = ESM_Model()
    embedding_dict = get_embedding_dict(sequence_dict, mod)
    score_df, subseq_df, pos_df = run_pairwise_kmer_emb_aln(
        reference_id,
        embedding_dict,
        k,
    )
    return score_df, subseq_df, pos_df

In [7]:
score_df, subseq_df, pos_df = main(seq_dict, reference_id, k)

In [8]:
pos_df

,reference_kmer,7757_0:00087f,42100_0:0032b1,8255_0:002868,215358_0:000d84,13616_0:00197d,336983_0:0004f6
0,FAEAWKAQFPDSEP,0,0,0,0,0,0
1,AEAWKAQFPDSEPP,1,1,1,1,1,14
2,EAWKAQFPDSEPPR,2,2,2,2,2,15
3,AWKAQFPDSEPPRM,3,3,3,3,3,16
4,WKAQFPDSEPPRME,4,4,4,4,4,17
...,...,...,...,...,...,...,...
465,GSRDALVSGALEST,275,469,496,395,438,59
466,SRDALVSGALESTK,276,470,497,391,433,60
467,RDALVSGALESTKA,277,471,498,397,340,25
468,DALVSGALESTKAS,278,472,499,398,341,26


In [9]:
torch.cuda.empty_cache()